In [7]:
from transformers import AutoTokenizer, AutoModel

In [8]:
model_name = "/data/models/google-bert--bert-base-chinese"

In [18]:
from transformers import pipeline

In [20]:
fill_mask = pipeline(task="fill-mask", model=model_name)

Some weights of the model checkpoint at /data/models/google-bert--bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [21]:
text = "人民是[MASK]可战胜的"
fill_mask(text, top_k=1)

[{'score': 0.9203749299049377,
  'token': 679,
  'token_str': '不',
  'sequence': '人 民 是 不 可 战 胜 的'}]

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [10]:
sentence = "美国的首都是华盛顿特区"
tokens = tokenizer.tokenize(sentence)
print(tokens)

['美', '国', '的', '首', '都', '是', '华', '盛', '顿', '特', '区']


In [12]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

[5401, 1744, 4638, 7674, 6963, 3221, 1290, 4670, 7561, 4294, 1277]


In [13]:
token_ids_e2e = tokenizer.encode(sentence)
print(token_ids_e2e)

[101, 5401, 1744, 4638, 7674, 6963, 3221, 1290, 4670, 7561, 4294, 1277, 102]


In [14]:
tokenizer.decode(token_ids_e2e)

'[CLS] 美 国 的 首 都 是 华 盛 顿 特 区 [SEP]'

In [15]:
sentence_batch = ["美国的首都是华盛顿特区", "中国的首都是北京"]
token_ids_batch = tokenizer.encode(sentence_batch)
tokenizer.decode(token_ids_batch)

'[CLS] 美 国 的 首 都 是 华 盛 顿 特 区 [SEP] 中 国 的 首 都 是 北 京 [SEP]'

In [26]:
embedding_batch = tokenizer("美国的首都是华盛顿特区", "中国的首都是北京")
print(embedding_batch)

{'input_ids': [101, 5401, 1744, 4638, 7674, 6963, 3221, 1290, 4670, 7561, 4294, 1277, 102, 704, 1744, 4638, 7674, 6963, 3221, 1266, 776, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [27]:
for key, value in embedding_batch.items():
    print(f"{key}: {value}\n")

input_ids: [101, 5401, 1744, 4638, 7674, 6963, 3221, 1290, 4670, 7561, 4294, 1277, 102, 704, 1744, 4638, 7674, 6963, 3221, 1266, 776, 102]

token_type_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]

attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]



In [28]:
len(tokenizer.vocab.keys())

21128

In [29]:
from itertools import islice

for key, value in islice(tokenizer.vocab.items(), 10):
    print(f"{key}: {value}")

出: 1139
辍: 6780
活: 3833
##增: 14929
313: 11622
猶: 4348
##伐: 13884
圆: 1749
泯: 3804
獲: 4363


In [30]:
new_tokens = ["天干","地支"]
new_tokens = set(new_tokens) - set(tokenizer.vocab.keys())
new_tokens

{'地支', '天干'}

In [31]:
tokenizer.add_tokens(list(new_tokens))

2

In [32]:
len(tokenizer.vocab.keys())

21130

In [ ]:
tokenizer.save_pretrained("path..")
model.save_pretrained("path...")

In [33]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
import evaluate
metric = evaluate.load("/data/models/evaluate-0.4.5/metrics/accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir=model_dir,
                                  per_device_train_batch_size=16,
                                  num_train_epochs=5,
                                  logging_steps=50,
                                  save_total_limit=5)

training_args = TrainingArguments(output_dir=model_dir,
                                  #evaluation_strategy="epoch", 
                                  eval_strategy="epoch",
                                  per_device_train_batch_size=16,
                                  num_train_epochs=3,
                                  logging_steps=30)

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
small_test_dataset = tokenized_datasets["test"].shuffle(seed=66).select(range(10))

In [ ]:
trainer.evaluate(small_test_dataset)